<a href="https://colab.research.google.com/github/dine1717/END2/blob/session7/Session_%26_Assignment_2_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.data import Field, BucketIterator,TabularDataset

import spacy
import numpy as np

import random
import math
import time

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
#!pip install spacy --upgrade

In [52]:
%%bash
python -m spacy download en
# python -m spacy download de

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2021-06-21 05:15:16.727587: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Data Preprocessing

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import pandas as pd
import glob

In [6]:
ls

drive/  sample_data/


In [13]:
cd /content/drive/MyDrive/Colab\ Notebooks/END6

/content/drive/MyDrive/Colab Notebooks/END6


In [14]:
PATH='/content/drive/MyDrive/Colab Notebooks/END6'

In [31]:
all_files = glob.glob(PATH + "/*.txt")
df=[]

for filename in all_files:
  data=pd.read_csv(filename,index_col=None,header=0,sep='\t',encoding='ISO-8859-1')
  print(f'{filename} in {data.shape}')
  df.append(data)

/content/drive/MyDrive/Colab Notebooks/END6/question_answer_pairs_s08.txt in (1715, 6)
/content/drive/MyDrive/Colab Notebooks/END6/question_answer_pairs_s09.txt in (825, 6)
/content/drive/MyDrive/Colab Notebooks/END6/question_answer_pairs_s10.txt in (1458, 6)


In [33]:
data_all = pd.concat(df,axis=0,ignore_index=True)

In [37]:
data_all.shape

(3998, 6)

In [36]:
data_all.sample(5)

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
268,Calvin_Coolidge,What Retrieved on May?,NaN,NaN,NaN,data/set3/a9
2944,Drum,Is the drum a member of the percussion group?,yes,easy,easy,data/set2/a4
3555,Nikola_Tesla,Was Nikola Tesla a vegetarian?,Yes,hard,easy,data/set4/a3
1619,Uruguay,Is uruguay's landscape mountainous?,not really?,hard,too hard,data/set2/a9
3716,San_Francisco,How large is the population of San Francisco?,San Francisco has an estimated population of 8...,medium,medium,data/set3/a8


In [38]:
data_all.isnull().sum()

ArticleTitle                  0
Question                     37
Answer                      576
DifficultyFromQuestioner    955
DifficultyFromAnswerer      580
ArticleFile                   2
dtype: int64

### Remove null or Nan  records from Question and Answer 

In [39]:
data_all.dropna(subset=['Question','Answer'],how='all',inplace=True)

In [43]:
data_all.dropna(subset=['Answer'],how='all',inplace=True)

In [44]:
data_all.isnull().sum()

ArticleTitle                  0
Question                      0
Answer                        0
DifficultyFromQuestioner    688
DifficultyFromAnswerer        5
ArticleFile                   2
dtype: int64

In [63]:
data_all.to_csv("Final_data.tsv",sep='\t')

In [54]:
spacy_en = spacy.load('en_core_web_sm')

In [55]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [57]:
data_all.sample(1)

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
1867,Cello,What are the two materials that bows are made ...,"Traditionally, bows are made from pernambuco o...",medium,medium,data/set2/a9


In [58]:
SRC = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

fields = {'Question' : ('src', SRC),
          'Answer' : ('trg', TRG)}

In [64]:
qa_data  = TabularDataset(path='Final_data.tsv',
                          format='tsv',
                          fields=fields
                           )

In [68]:
vars(qa_data.examples[1868])

{'src': ['did',
  'newton',
  'reject',
  'the',
  'church',
  "'s",
  'doctrine',
  'of',
  'the',
  'trinity',
  '?'],
 'trg': ['newton',
  'may',
  'have',
  'rejected',
  'the',
  'church',
  "'s",
  'doctrine',
  'of',
  'the',
  'trinity',
  '.']}

In [69]:
train_data, test_data = qa_data.split([0.7,0.3])

In [70]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 2395
Number of testing examples: 1027


In [71]:
print(vars(train_data.examples[0]))

{'src': ['how', 'does', 'poverty', 'in', 'san', 'francisco', 'compare', 'to', 'the', 'nation', '-', 'wide', 'average', '?'], 'trg': ['san', 'francisco', "'s", 'poverty', 'rate', 'is', 'lower', 'than', 'the', 'national', 'average', '.']}


In [73]:
print(vars(test_data.examples[0]))

{'src': ['where', 'is', 'the', 'most', 'densely', 'populated', 'part', 'of', 'canada', '?'], 'trg': ['the', 'most', 'densely', 'populated', 'part', 'of', 'the', 'country', 'is', 'the', 'quebec', 'city', '-', 'windsor', 'corridor', 'along', 'the', 'great', 'lakes', 'and', 'saint', 'lawrence', 'river', 'in', 'the', 'southeast', '.']}


In [115]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [116]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 1979
Unique tokens in target (en) vocabulary: 1281


In [117]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [118]:
BATCH_SIZE = 128

train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_key = lambda x: len(x.src),
    sort_within_batch=True,
    device = device)

In [119]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [120]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [121]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [122]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [123]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(1979, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(1281, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=1281, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [124]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 8,848,129 trainable parameters


In [125]:
optimizer = optim.Adam(model.parameters())

In [126]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [127]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [128]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [129]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [130]:
N_EPOCHS = 10
CLIP = 1

best_test_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    test_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), 'qa1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Test. Loss: {test_loss:.3f} |  Test. PPL: {math.exp(test_loss):7.3f}')

Epoch: 01 | Time: 0m 2s
	Train Loss: 5.190 | Train PPL: 179.407
	 Test. Loss: 3.495 |  Test. PPL:  32.953
Epoch: 02 | Time: 0m 2s
	Train Loss: 4.407 | Train PPL:  82.007
	 Test. Loss: 3.492 |  Test. PPL:  32.843
Epoch: 03 | Time: 0m 2s
	Train Loss: 4.314 | Train PPL:  74.717
	 Test. Loss: 3.448 |  Test. PPL:  31.430
Epoch: 04 | Time: 0m 2s
	Train Loss: 4.253 | Train PPL:  70.289
	 Test. Loss: 3.455 |  Test. PPL:  31.650
Epoch: 05 | Time: 0m 2s
	Train Loss: 4.200 | Train PPL:  66.714
	 Test. Loss: 3.482 |  Test. PPL:  32.527
Epoch: 06 | Time: 0m 2s
	Train Loss: 4.123 | Train PPL:  61.774
	 Test. Loss: 3.445 |  Test. PPL:  31.346
Epoch: 07 | Time: 0m 2s
	Train Loss: 4.074 | Train PPL:  58.803
	 Test. Loss: 3.350 |  Test. PPL:  28.504
Epoch: 08 | Time: 0m 2s
	Train Loss: 3.997 | Train PPL:  54.417
	 Test. Loss: 3.329 |  Test. PPL:  27.912
Epoch: 09 | Time: 0m 2s
	Train Loss: 3.926 | Train PPL:  50.680
	 Test. Loss: 3.379 |  Test. PPL:  29.350
Epoch: 10 | Time: 0m 2s
	Train Loss: 3.919 | T

In [132]:
model.load_state_dict(torch.load('qa1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.326 | Test PPL:  27.816 |


In [145]:
def show_predictions(question, answer, prediction, example_id):
  model_prediction = " ".join([SRC.vocab.itos[x] for x in torch.argmax(prediction[:, example_id, :], axis = 1).cpu()])

  print("Question:\n")
  print(" ".join([SRC.vocab.itos[x] for x in question[:, example_id].cpu()]))
    

  print("\nActual Answer: \n")
  print(" ".join([SRC.vocab.itos[x] for x in answer[:, example_id].cpu()]))

  print("\nPredicted Answer: \n")
  print(model_prediction)

In [150]:
# example_number = 5
# test_iterator_list = list(test_iterator)
# batch = test_iterator_list[example_number]
# question = batch.src
# answer = batch.trg
# prediction = model(question, answer, 5)

In [151]:
# show_predictions(question, answer, prediction, example_number)
